In [1]:
%matplotlib inline

In [2]:
import pandas as pd
from os.path import join, exists
from os import mkdir
from glob import glob
import xml.etree.ElementTree as ET
import numpy as np
from scipy import stats
from tqdm import tqdm
import statsmodels.api as sm
import seaborn as sns
import matplotlib.pyplot as plt

from statsmodels.stats.multitest import multipletests

from itertools import combinations
import json

pd.set_option('display.max_rows', None, 'display.max_columns', None)

# Correlation study data

In [3]:
def export_df(directory, psych_test='MMSE'):
    XML_COLS = ['Age', 'Sex', 'APOE_A1', 'APOE_A2', 'MMSE', 'NPIQ']

    stats_df = pd.read_csv(join(directory, 'stats', 'output_'+psych_test.lower()+'.csv'))

    if stats_df.columns[0] != 'PET_ID':
        stats_df.drop(stats_df.columns[0], axis=1, inplace=True)

    stats_df['Scan_Date'] = stats_df['PET_ID'].apply(lambda id: id.split('~')[1].split('_')[0])
    stats_df['PET_ID'] = stats_df['PET_ID'].apply(lambda id: id.split('~')[0] + '-' + id.split('~')[-1])
    stats_df['Subject_ID'] = stats_df['PET_ID'].apply(lambda id: id.split('-')[0])

    col_list = list(stats_df.columns)
    new_col_list = col_list[0:1] + col_list[2:] + list(col_list[1:2])
    stats_df = stats_df[new_col_list]

    for xml_col in XML_COLS:
        stats_df[xml_col] = None
    
    col_list = list(stats_df.columns)
    new_col_list = col_list[0:1] + col_list[-9:] + col_list[1:-9]
    stats_df = stats_df[new_col_list]
   
    metadata_dir = join(directory, 'Metadata', 'ADNI')
    xml_files = glob(join(metadata_dir, '*.xml'))

    tree = None

    for xml_file in xml_files:
        xml_file_name = xml_file.split('/')[-1]
        subject_id = '_'.join(xml_file_name.split('_')[1:4])
        other_id = xml_file_name.split('_')[-1].split('.')[0]
        unique_id = subject_id + '-' + other_id

        if len(stats_df.loc[stats_df['PET_ID'] == unique_id].index.values) == 0:
            continue

        tree = ET.parse(xml_file)
        root = tree.getroot()

        if len(root.findall(".//subjectAge")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'Age'] = root.findall(".//subjectAge")[0].text
        if len(root.findall(".//subjectSex")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'Sex'] = root.findall(".//subjectSex")[0].text
        if len(root.findall(".//subjectInfo[@item='APOE A1']")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'APOE_A1'] = root.findall(".//subjectInfo[@item='APOE A1']")[0].text
        if len(root.findall(".//subjectInfo[@item='APOE A2']")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'APOE_A2'] = root.findall(".//subjectInfo[@item='APOE A2']")[0].text
        if len(root.findall(".//assessmentScore[@attribute='MMSCORE']")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'MMSE'] = root.findall(".//assessmentScore[@attribute='MMSCORE']")[0].text
        if len(root.findall(".//assessmentScore[@attribute='NPISCORE']")) > 0:
            stats_df.at[stats_df.loc[stats_df['PET_ID'] == unique_id].index.values[0], 'NPIQ'] = root.findall(".//assessmentScore[@attribute='NPISCORE']")[0].text
    
    object_cols = ['Age', 'APOE_A1', "APOE_A2", 'MMSE', 'NPIQ']
    for object_col in object_cols:
        stats_df[object_col] = stats_df[object_col].apply(pd.to_numeric, errors='coerce')
    
    rois_df = stats_df.drop(stats_df.columns[0:10], axis=1, inplace=False)
    corr_data = {psych_test+'_Corr': [], psych_test+'_p_value': []}

    for roi in rois_df:
        x, y = stats_df[psych_test].values, rois_df[roi].values
        nans = np.logical_or(np.isnan(x), np.isnan(y))
        try:
            score_corr, score_p = stats.pearsonr(x[~nans], y[~nans])
            corr_data[psych_test+'_Corr'].append(score_corr)
            corr_data[psych_test+'_p_value'].append(score_p)
        except ValueError:
            print(directory+' has fewer than 2 values in '+psych_test)
            return (stats_df, None)
    
    corr_data_df = pd.DataFrame.from_dict(corr_data)
    corr_data_df['ROI'] = rois_df.columns
    corr_data_df.set_index('ROI', inplace=True)
    corr_data_df.sort_values([psych_test+'_Corr'], ignore_index=False, inplace=True)
    
    return (stats_df, corr_data_df)

In [4]:
directories = glob(join('..', 'New_Data', '*', '*'))

# # stats_df.fillna(value=np.nan, inplace=True)

# object_cols = ['Age', 'APOE_A1', "APOE_A2", 'MMSE', 'NPIQ']
# for object_col in object_cols:
#     stats_df[object_col] = stats_df[object_col].apply(pd.to_numeric, errors='coerce')

# stats_df.drop(stats_df.columns[0:10], axis=1, inplace=True)
# stats_df
# count = 0

# with tqdm(total=len(directories), desc='Directories analyzed') as pbar:
#     for directory in directories:
#         stats_df, corr_data_df = export_df(directory)
#         _, npiq_corr_data_df = export_df(directory, psych_test='NPIQ')

#         stats_df.to_csv(join(directory, 'stats', 'summary.csv'), index=False)
#         if corr_data_df is not None:
#             corr_data_df.to_csv(join(directory, 'stats', 'mmse_corr.csv'))
#         if npiq_corr_data_df is not None:
#             npiq_corr_data_df.to_csv(join(directory, 'stats', 'npiq_corr.csv'))

#         pbar.update()

# RoI ranking generated from Influential nodes data

In [5]:
def calculate_roi_ranks(directory):
    influential_df = pd.read_csv(join(directory, 'stats', 'influential.csv'))
    influential_df.drop(influential_df.columns[0], axis=1, inplace=True)

    roi_score = {}
    roi_occurrences = {}

    for ind in influential_df.index:
        rois = influential_df['Influential node values'][ind]
        rois = rois.split(',')
        rois = [roi.strip() for roi in rois]

        for i in range(len(rois)):
            if rois[i] in roi_score:
                roi_score.update({rois[i]: roi_score[rois[i]]+i+1})
                roi_occurrences.update({rois[i]: roi_occurrences[rois[i]]+1})
            else:
                roi_score[rois[i]] = i+1
                roi_occurrences[rois[i]] = 1

    ranking_df_dict = {'ROI': [roi for roi in roi_score], \
                       'Occurrences': [roi_occurrences[roi] for roi in roi_score], \
                       'Rank': [roi_score[roi]//roi_occurrences[roi] for roi in roi_score]}
    ranking_df = pd.DataFrame.from_dict(ranking_df_dict)
    ranking_df.sort_values(['Rank', 'Occurrences'], ascending=[True, False], ignore_index=True, inplace=True)

    ranking_df.to_csv(join(directory, 'stats', 'roi_ranking.csv'), index=False)

In [6]:
# directories = glob(join('..', 'New_Data', '*', '*'))

# # directory = '../New_Data/MCI/AV45'
# # calculate_roi_ranks(directory)

# for directory in directories:
#     calculate_roi_ranks(directory)
#     # calculate_roi_ranks(directory, psych_test='NPIQ')

# ANOVA

In [7]:
def calculate_anova(dir1, dir2, dir3):
    radioisotopes = ['AV45', 'FDG', 'PiB']
    for radioisotope in radioisotopes:
        mmse_cn_df = pd.read_csv(join(dir1, radioisotope, 'stats', 'output_mmse.csv'))
        npiq_cn_df = pd.read_csv(join(dir1, radioisotope, 'stats', 'output_npiq.csv'))
        cn_df = pd.concat([mmse_cn_df, npiq_cn_df], axis=1)
        cn_df = cn_df.loc[:,~cn_df.columns.duplicated()]
        cn_df.drop([cn_df.columns[i] for i in range(2)], axis=1, inplace=True)

        mmse_mci_df = pd.read_csv(join(dir2, radioisotope, 'stats', 'output_mmse.csv'))
        npiq_mci_df = pd.read_csv(join(dir2, radioisotope, 'stats', 'output_npiq.csv'))
        mci_df = pd.concat([mmse_mci_df, npiq_mci_df], axis=1)
        mci_df = mci_df.loc[:,~mci_df.columns.duplicated()]
        mci_df.drop([mci_df.columns[i] for i in range(2)], axis=1, inplace=True)

        mmse_ad_df = pd.read_csv(join(dir3, radioisotope, 'stats', 'output_mmse.csv'))
        npiq_ad_df = pd.read_csv(join(dir3, radioisotope, 'stats', 'output_npiq.csv'))
        ad_df = pd.concat([mmse_ad_df, npiq_ad_df], axis=1)
        ad_df = ad_df.loc[:,~ad_df.columns.duplicated()]
        ad_df.drop([ad_df.columns[i] for i in range(2)], axis=1, inplace=True)

        anova_df_dict = {'ROI': [], 'f_value': [], 'p_value': []}
        for roi in cn_df:
            anova_df_dict['ROI'].append(roi)
            f, p = stats.f_oneway(cn_df[roi], mci_df[roi], ad_df[roi])
            anova_df_dict['f_value'].append(f)
            anova_df_dict['p_value'].append(p)
        
        anova_df = pd.DataFrame.from_dict(anova_df_dict)
        anova_df.set_index("ROI", inplace=True)
        anova_df.sort_values(['f_value', 'p_value'], ascending=[False, True], inplace=True)

        if not exists(join('..', 'New_Data', 'stats')):
            mkdir(join('..', 'New_Data', 'stats'))
        anova_df.to_csv(join('..', 'New_Data', 'stats', 'anova_'+radioisotope.lower()+'.csv'), index=True)

In [8]:
# directories = glob(join('..', 'New_Data', '*'))

directory2 = '../New_Data/MCI'
directory1 = '../New_Data/CN'
directory3 = '../New_Data/AD'

# # for directory in directories:
# calculate_anova(directory1, directory2, directory3)
# # calculate_anova(directory1, directory2, directory3, psych_test='NPIQ')

# MLR

In [9]:
def perform_mlr(radioisotope, psych_test='MMSE'):
    indep_df = pd.read_csv(join(dataset_path, 'AD', radioisotope, 'stats', 'output_'+psych_test.lower()+'.csv'))
    indep_df = pd.concat([indep_df, pd.read_csv(join(dataset_path, 'MCI', radioisotope, 'stats', 'output_'+psych_test.lower()+'.csv'))], ignore_index=True)
    indep_df = pd.concat([indep_df, pd.read_csv(join(dataset_path, 'CN', radioisotope, 'stats', 'output_'+psych_test.lower()+'.csv'))], ignore_index=True)
    indep_df.drop([indep_df.columns[i] for i in range(2)], axis=1, inplace=True)

    target_df = pd.read_csv(join(dataset_path, 'AD', radioisotope, 'stats', 'summary.csv'))
    target_df = pd.concat([target_df, pd.read_csv(join(dataset_path, 'MCI', radioisotope, 'stats', 'summary.csv'))], ignore_index=True)
    target_df = pd.concat([target_df, pd.read_csv(join(dataset_path, 'CN', radioisotope, 'stats', 'summary.csv'))], ignore_index=True)

    X = indep_df
    y = target_df[psych_test]

    model = sm.OLS(y, X, missing='drop').fit()
    results_summary = model.summary()

    results_as_html_0 = results_summary.tables[0].as_html()
    res_df_0 = pd.read_html(results_as_html_0, header=None, index_col=0)[0]

    results_as_html_1 = results_summary.tables[1].as_html()
    res_df_1 = pd.read_html(results_as_html_1, header=0, index_col=0)[0]
    res_df_1.sort_values(['t'], inplace=True)

    res_df_0.to_csv(join('..', 'New_Data', 'stats', 'mlr_'+radioisotope.lower()+'_'+psych_test.lower()+'_model_summary.csv'))
    res_df_1.to_csv(join('..', 'New_Data', 'stats', 'mlr_'+radioisotope.lower()+'_'+psych_test.lower()+'_model_coeffs.csv'))
    # return res_df_1

In [10]:
dataset_path = join('..', 'New_Data')

radioisotopes = ['AV45', 'FDG', 'PiB']

# df = None
# for radioisotope in radioisotopes[1:2]:
#     # df = perform_mlr(radioisotope)
#     perform_mlr(radioisotope, psych_test='NPIQ')

# df.rename(columns={'P>|t|':'p'}, inplace=True)

# pvals = df['p'].tolist()
# df

# ANOVA results analysis

In [11]:
# radioisotopes = ['AV45', 'FDG', 'PiB']
# radioisotope_critical_f = {'AV45': 3.005, 'FDG': 3.00053, 'PiB': 3.042}
# stats_path = join('..', 'New_Data', 'stats')

# anova_dfs = {radioisotope: pd.read_csv(join(stats_path, 'anova_'+radioisotope.lower()+'.csv')) for radioisotope in radioisotopes}
# anova_dfs.update({radioisotope: anova_dfs[radioisotope].loc[anova_dfs[radioisotope]['f_value']>radioisotope_critical_f[radioisotope]] for radioisotope in radioisotopes})

# av45_rois = set(anova_dfs['AV45']['ROI'].tolist())
# fdg_rois = set(anova_dfs['FDG']['ROI'].tolist())
# pib_rois = set(anova_dfs['PiB']['ROI'].tolist())

In [12]:
# int1 = av45_rois.intersection(fdg_rois)
# int1

In [13]:
# int2 = av45_rois.intersection(pib_rois)
# int2

In [14]:
# int3 = fdg_rois.intersection(pib_rois)
# int3

### We find that there are no nodes in common across the 3 radioisotopes over the critical F-values for each of the 3 radioisotopes

# Adjacency matrix heatmap

In [15]:
# scan = '131_S_0497~2006-06-28_12_44_51.0~I17585'
# adj_mat = np.load(join('..', 'New_Data', 'AD', 'FDG', scan, 'adj_mat.npy'))
# np.fill_diagonal(adj_mat, 0)

# heat_map = sns.heatmap(adj_mat, xticklabels=False, yticklabels=False)
# plt.show()

In [16]:
# scan = '131_S_0497~2006-06-28_12_44_51.0~I17585'
# adj_mat = np.load(join('..', 'New_Data', 'AD', 'FDG', scan, 'adj_mat_thresh.npy'))
# np.fill_diagonal(adj_mat, 0)

# heat_map = sns.heatmap(adj_mat, xticklabels=False, yticklabels=False)
# plt.show()

In [17]:
# import networkx as nx

In [18]:
# net = nx.from_numpy_matrix(adj_mat)
# net.number_of_edges()

In [19]:
# net_thresh = nx.from_numpy_matrix(np.load(join('..', 'New_Data', 'AD', 'FDG', scan, 'adj_mat_thresh.npy')))
# net_thresh.number_of_edges()

# ANOVA - Influential ranking comparison

In [20]:
# radioisotopes = ['AV45', 'FDG', 'PiB']
# radioisotope_critical_f = {'AV45': 3.005, 'FDG': 3.00053, 'PiB': 3.042}
# stats_path = join('..', 'New_Data', 'stats')

# anova_dfs = {radioisotope: pd.read_csv(join(stats_path, 'anova_'+radioisotope.lower()+'.csv')) for radioisotope in radioisotopes}
# anova_dfs.update({radioisotope: anova_dfs[radioisotope].loc[anova_dfs[radioisotope]['f_value']>radioisotope_critical_f[radioisotope]] for radioisotope in radioisotopes})

# av45_rois = set(anova_dfs['AV45']['ROI'].tolist())
# fdg_rois = set(anova_dfs['FDG']['ROI'].tolist())
# pib_rois = set(anova_dfs['PiB']['ROI'].tolist())

In [21]:
# dataset_path = join('..', 'New_Data')

# diagnoses_paths = [join(dataset_path, diagnosis) for diagnosis in ['AD', 'CN', 'MCI']]

# for diagnosis_path in diagnoses_paths:
#     print(diagnosis_path.split('/')[-1])
#     for radioisotope in radioisotopes:
#         print(radioisotope)
#         rois = anova_dfs[radioisotope]['ROI'].tolist()
#         ranking_df = pd.read_csv(join(diagnosis_path, radioisotope, 'stats', 'roi_ranking.csv'))
#         for roi in rois:
#             roi = roi.replace(' ', '.')
#             # print(roi)
#             row = ranking_df.loc[ranking_df['ROI'] == roi]
#             # print(row)
#             ind = ranking_df.loc[ranking_df['ROI'] == roi].index.values[0]
#             print('ROI: ', row.iloc[0]['ROI'], '\t', 'Rank: ', row.iloc[0]['Rank'], '\t', 'Relative list rank: ', ind + 1)
#     print('\n')

In [22]:
def circos(ranklist1_name, ranklist2_name, ranklist1, ranklist2):
    header = ranklist1['ROI'].tolist()
    header = {'One_'+str(x+1): header[x] for x in range(len(header))}
    ind = ranklist2['ROI'].tolist()
    ind = {'Two_'+str(x+1): ind[x] for x in range(len(ind))}

    header_keys = list(header.keys())
    ind_keys = list(ind.keys())

    # print(header_keys)
    # print(ind_keys)

    df = pd.DataFrame(index=ind_keys, columns=header_keys)

    # return df
    for i in range(30):
        df.iloc[i] = [abs(x - i) if header[header_keys[x]] == ind[ind_keys[i]] else 0 for x in range(len(header_keys))]

    df = df[(df.T != 0).any()]
    df = df.loc[:, (df != 0).any(axis=0)]
    df = df.sample(frac = 1) 

    df.to_csv(join('circos_files', ranklist1_name + '_' + ranklist2_name+'.tsv'), sep='\t')
    with open(join('circos_files', ranklist1_name + '_' + ranklist2_name+'.tsv'), 'r+') as f:
        content = f.read()
        f.seek(0, 0)
        f.write('data' + content)
        f.close()
    json.dump(header, open(join('circos_files', ranklist1_name + '_' + ranklist2_name + '_ranklist1_legend.json'), 'w'))
    json.dump(ind, open(join('circos_files', ranklist1_name + '_' + ranklist2_name + '_ranklist2_legend.json'), 'w'))

In [23]:
# ranklist1 = pd.read_csv(join('..', 'New_Data', 'AD', 'FDG', 'stats', 'roi_ranking.csv'))
# ranklist1 = ranklist1.head(30)

In [24]:
# ranklist2 = pd.read_csv(join('..', 'New_Data', 'AD', 'AV45', 'stats', 'roi_ranking.csv'))
# ranklist2 = ranklist2.head(30)

In [25]:
# header = ranklist1['ROI'].tolist()
# header = ['One_'+elem.replace('.', '_').replace('/', 'x').replace('-', '_').replace("'", '_') for elem in header]
# ind = ranklist2['ROI'].tolist()
# ind = ['Two_'+elem.replace('.', '_').replace('/', '_').replace('-', '_').replace("'", '_') for elem in ind]

# df = pd.DataFrame(index=ind, columns=header)
# df

In [26]:
# for i in range(30):
#     df.iloc[i] = [abs(x - i) if header[x][4:] == ind[i][4:] else 0 for x in range(len(header))]

# df

In [27]:
# df = df[(df.T != 0).any()]
# df = df.loc[:, (df != 0).any(axis=0)]
# df = df.sample(frac = 1) 
# df

In [28]:
# df.to_csv('temp.tsv', sep='\t')

In [29]:
dataset_path = join('..', 'New_Data')

diagnoses_paths = glob(join(dataset_path, '*'))
radioisotopes = ['AV45', 'FDG', 'PiB']
ranklists = dict()

for diagnosis_path in diagnoses_paths:
    diagnosis = diagnosis_path.split('/')[-1]
    for radioisotope in radioisotopes:
        ranklists[diagnosis+'_'+radioisotope] = pd.read_csv(join(diagnosis_path, radioisotope, 'stats', 'roi_ranking.csv')).head(30)

li = [(key, ranklists[key]) for key in ranklists]
li2 = list(combinations(li, 2))

# print(li2)

for tup in li2:
    tup1 = tup[0]
    tup2 = tup[1]

    tup1_name = tup1[0]
    tup2_name = tup2[0]

    tup1_list = tup1[1]
    tup2_list = tup2[1]

    # print(tup1_name, ' ', tup2_name, end=': ')
    circos(tup1_name, tup2_name, tup1_list, tup2_list)
    # print(df)
    # break
